# Save the Model

To save this model so that you can use it from various locations, including other notebooks or the model server, upload it to s3-compatible storage.

## Install the required packages and define a function for the upload

In [ ]:
!pip install boto3 botocore lakefs==0.7.1

### Define lakeFS Repository

In [ ]:
import os
import lakefs
repo_name = os.environ.get('LAKEFS_REPO_NAME')

mainBranch = "main"
trainingBranch = "train01"

repo = lakefs.Repository(repo_name)
print(repo)

In [ ]:
import boto3
import botocore

aws_access_key_id = os.environ.get('LAKECTL_CREDENTIALS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('LAKECTL_CREDENTIALS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('LAKECTL_SERVER_ENDPOINT_URL')
region_name = os.environ.get('LAKEFS_DEFAULT_REGION')
bucket_name = os.environ.get('LAKEFS_REPO_NAME')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or data connection variables are empty.  "
                     "Please check your data connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
            num_files += 1
    return num_files


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

## Verify the upload

In your S3 bucket, under the `models` upload prefix, run the `list_object` command. As best practice, to avoid mixing up model files, keep only one model and its required files in a given prefix or directory. This practice allows you to download and serve a directory with all the files that a model requires. 

If this is the first time running the code, this cell will have no output.

If you've already uploaded your model, you should see this output: `models/fraud/1/model.onnx`


In [ ]:
list_objects(f"{trainingBranch}/models")

## Upload the model to the training branch in lakeFS and check again

Use the function to upload the `models` folder in a rescursive fashion:

In [ ]:
local_models_directory = "models"

if not os.path.isdir(local_models_directory):
    raise ValueError(f"The directory '{local_models_directory}' does not exist.  "
                     "Did you finish training the model in the previous notebook?")

num_files = upload_directory_to_s3("models", f"{trainingBranch}/models")

if num_files == 0:
    raise ValueError("No files uploaded.  Did you finish training and "
                     "saving the model to the \"models\" directory?  "
                     "Check for \"models/fraud/1/model.onnx\"")


To confirm this worked, run the `list_objects` function again:

In [ ]:
list_objects(f"{trainingBranch}/models")

## Commit changes in lakeFS repository

In [ ]:
branchTraining = repo.branch(trainingBranch)
ref = branchTraining.commit(message='Uploaded data, artifacts and model')
print(ref.get_commit())

### Next Step

Now that you've saved the model to s3 storage, you can refer to the model by using the same data connection to serve the model as an API.
